In [16]:
# Import Module

import tkinter as tk
import pyodbc
from tkinter import ttk  
from tkinter import messagebox
from tkinter.filedialog import asksaveasfilename
import csv
#import unicodecsv as csv
import sys
import codecs
import tkinter.font as font

LARGEFONT =("Verdana", 35)
SMALLFONT=("Verdana", 13)

#tạo connect SQL server
conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=MON-MON-PC;DATABASE=QuanLyGiangVien;Trusted_Connection=yes;')
curѕor = conn.cursor()

class baiCuoiKhoa(tk.Tk):
    # __init__ hàm khởi tạo
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.title("HỆ THỐNG QUẢN LÝ GIẢNG VIÊN")
  
        # creating a container
        container = tk.Frame(self) 
        container.pack(side = "top", fill = "both", expand = True)
  
        container.grid_rowconfigure(0, weight = 1)
        container.grid_columnconfigure(0, weight = 1)
  
        # initializing frames to an empty array
        self.frames = {} 
  
        # iterating through a tuple consisting
        # of the different page layouts
        for F in (GiangVienPage, MonHocPage, GiangVienMonHocPage):
            frame = F(container, self)
  
            # initializing frame of that object from
            # GiangVienPage, MonHocPage, GiangVienMonHocPage respectively with
            # for loop
            self.frames[F] = frame
            frame.grid(row = 0, column = 0, sticky ="nsew")
  
        self.show_frame(GiangVienPage)
  
    # to display the current frame passed as
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

# trang thông tin GV - GiangVienPage
class GiangVienPage(tk.Frame):
    
    def __init__(self, parent, controller):
        self.parent = parent
        tk.Frame.__init__(self, parent)
        
        # khai báo biến
        MaGV = tk.StringVar()
        HoDem = tk.StringVar()
        Ten = tk.StringVar()
        SDT = tk.StringVar()
        CCCD = tk.StringVar()
        DiaChi = tk.StringVar()
        NgaySinh = tk.StringVar()  
        HocVi = tk.StringVar() 
        NghiHuu = tk.BooleanVar()
        GioiTinh = tk.BooleanVar()
        IDKhoa = tk.IntVar()
        TuKhoa = tk.StringVar()
        
        #hiển thị danh sách GV trên lưới
        def show_item():  
            dsGiangVien.delete(0,tk.END)
            dsGiangVien.insert(tk.END, f' - Mã GV - Họ tên - Khoa CQ - Ngày sinh - Điện thoại')
            dsGVFull = curѕor.execute("EXEC dbo.SP_LoadGiangVien ?, null, null", (TuKhoa.get()))
            for c in dsGVFull.fetchall():
                dsGiangVien.insert(tk.END, f'{c[0]} - {c[1]} - {c[2]} {c[3]} - {c[4]} - {c[5]} - {c[6]}')
        
        #xóa dòng hiện tại trên lưới
        def remove_item():
            selection = dsGiangVien.curselection()
            if len(selection) > 0:
                item = dsGiangVien.get(selection[0])
                if item.split(' - ')[0] != 'Mã GV':
                    res = messagebox.askquestion('Xác nhận', 
                                                 'Bạn chắc chắn muốn xóa Giảng viên '+ item.split(' - ')[1] + ' - ' + item.split(' - ')[2] + '?')
                    if res == 'yes':
                        id = int(item.split(' - ')[0])
                        curѕor.execute("EXEC dbo.SP_XoaGiangVien ?", (id))
                        curѕor.commit()
                        show_item()
        
        #thêm hoặc cập nhật GV
        def add_item():
            if cboKhoa.get() == '' or MaGV.get() == '' or HoDem.get() == '' or Ten.get() == '':
                messagebox.showinfo("Warning", "Vui lòng chọn khoa và nhập đầy đủ mã/ họ tên GV")
            else:
                IDKhoa = ({i for i in lstTenKhoa if lstTenKhoa[i]==cboKhoa.get()}).pop()
                curѕor.execute("EXEC dbo.SP_CapNhatGiangVien ?,?,?,?,?,?,?,?,?,?,?", 
                               (MaGV.get(), HoDem.get(),Ten.get(),GioiTinh.get(),NgaySinh.get(),HocVi.get(),IDKhoa,CCCD.get(),SDT.get(), DiaChi.get(), NghiHuu.get()))
                curѕor.commit()
                show_item()
        
        #clear các control
        def reset_item():
            MaGV.set('')
            HoDem.set('')
            Ten.set('')
            SDT.set('')
            CCCD.set('')
            DiaChi.set('')
            NgaySinh.set('') 
            HocVi.set('')
            NghiHuu.set(False)
            GioiTinh.set(False)
            IDKhoa.set(0)
            cboKhoa.set('')
            TuKhoa.set('')
        #load thông tin dòng hiện tại lên các control
        def loadCurrent(event):
            if dsGiangVien.curselection() != ():
                curGV = dsGiangVien.get(dsGiangVien.curselection()).split(' - ')
                if curGV[1].strip()!="Mã GV":
                    gv = curѕor.execute("EXEC dbo.SP_LoadGiangVien null, null, ?", (curGV[0].strip())).fetchall()[0]
                    MaGV.set(gv[1])
                    HoDem.set(gv[2])
                    Ten.set(gv[3])
                    SDT.set(gv[6])
                    CCCD.set(gv[10])
                    DiaChi.set(gv[7])
                    NgaySinh.set(gv[5]) 
                    HocVi.set(gv[9])
                    NghiHuu.set(gv[12])
                    GioiTinh.set(gv[13])
                    IDKhoa.set(gv[11])
                    cboKhoa.set(gv[4])
        
        #xuất DS ra file CSV
        def ExportCSV():                
            try:
                name = asksaveasfilename()
                dsGVFull = curѕor.execute("EXEC dbo.SP_LoadGiangVien ?, null, null", (TuKhoa.get()))
            
                #with open('name.csv', 'w', encoding='utf-16') as csvfile:
                    #create = csv.writer(csvfile,quoting=csv.QUOTE_ALL)
                    #for c in dsGVFull.fetchall():
                        #csvfile.write(f'{c[0]}, {c[1]}, {c[2]} {c[3]}, {c[4]}, {c[5]}, {c[6]}\n')
            
                file = open (name + '.csv',"w",encoding='utf-16')
                create = csv.writer(file)
                for c in dsGVFull.fetchall():
                    file.write(f'{c[1]}, {c[2]} {c[3]}, {c[4]}, {c[5]}, {c[6]}\n')

                file.close()
                messagebox.showinfo("Information", f"Xuất file {name}.csv thành công")
            except Exception as e:
                messagebox.showinfo("Error!!!", str(e))
        
        #hiển thị 2 button chuyển đổi qua lại giữa 3 frame
        btnMH = ttk.Button(self, text ="Quản lý môn học", 
        command = lambda : controller.show_frame(MonHocPage))
        btnMH.grid(row = 0, column = 0, padx = 10, pady = 10)
        
        btnGVMH = ttk.Button(self, text ="Quản lý Giảng viên - môn học",
        command = lambda : controller.show_frame(GiangVienMonHocPage))
        btnGVMH.grid(row = 0, column = 1, padx = 10, pady = 10)
        
        #khai báo các control 
        tk.Label(self, text = 'Từ khóa (mã/ họ tên GV:)', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =1, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = TuKhoa).grid(row =1, column=1,columnspan = 3, padx = 10, pady = 10,sticky = tk.W+tk.E)
        tk.Button(self,text="Tìm kiếm",font=SMALLFONT,command=show_item).grid(row = 1, column = 4, padx = 10, pady = 10,sticky = tk.W)
    
        scroll_bar = tk.Scrollbar(self, orient=tk.VERTICAL)
        dsGiangVien = tk.Listbox(self, yscrollcommand=scroll_bar.set, height=12)
        scroll_bar.config(command=dsGiangVien.yview)
        scroll_bar.grid(row = 2, column = 4, padx = 10, pady = 10)
        dsGiangVien.grid(row = 2, column = 0, columnspan = 4, padx = 10, pady = 10,sticky = tk.W+tk.E)
        dsGiangVien.bind("<<ListboxSelect>>", loadCurrent)
        
        tk.Label(self, text = 'Chi tiết thêm/ cập nhật', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =3, column = 0, padx = 10, pady = 10)
        ttk.Separator(self, orient=tk.HORIZONTAL).grid(row=3,column=1,columnspan=4, padx=10, pady=2, sticky=tk.EW)
        
        tk.Label(self, text = 'Mã Giảng viên', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 4, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = MaGV,width=50).grid(row = 4, column = 1, padx = 10, pady = 10,sticky=tk.W)
        tk.Checkbutton(self, text = "Chấm dứt HĐ",font=SMALLFONT, variable=NghiHuu, onvalue = 1, offvalue = 0).grid(row = 4, column = 2, padx = 10, pady = 10,sticky=tk.W)
        
        tk.Label(self, text = 'Họ đệm', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 5, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = HoDem,width=50).grid(row = 5, column = 1, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Tên', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 5, column = 2, padx = 10, pady = 10)
        tk.Entry(self, textvariable = Ten,width=50).grid(row = 5, column = 3, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Ngày sinh', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 6, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = NgaySinh,width=50).grid(row = 6, column = 1, padx = 10, pady = 10)
        
        tk.Radiobutton(self, text = "Nam",font=SMALLFONT, variable=GioiTinh, value=0, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 6, column = 2, padx = 10, pady = 10)
        tk.Radiobutton(self, text = "Nữ",font=SMALLFONT, variable=GioiTinh, value=1, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 6, column = 3, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Học vị', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 7, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = HocVi,width=50).grid(row = 7, column = 1, padx = 10, pady = 10)
        
        tk.Label(self, text = 'CCCD', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 7, column = 2, padx = 10, pady = 10)
        tk.Entry(self, textvariable = CCCD,width=50).grid(row = 7, column = 3, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Số điện thoại', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 8, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = SDT,width=50).grid(row = 8, column = 1, padx = 10, pady = 10)
        
        tk.Label(self, text = 'Khoa', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 8, column = 2, padx = 10, pady = 10)
        
        query="SELECT DISTINCT TenKhoa, Id FROM dbo.Khoa"
                
        lstKhoa =conn.execute(query)
        lstTenKhoa = {}
        for columnrow in lstKhoa.fetchall():
            lstTenKhoa[columnrow[1]] = columnrow[0]
        khoa =tk.StringVar() 
        dsKhoa = list(lstTenKhoa.values())
        cboKhoa = ttk.Combobox(self, values=dsKhoa, textvariable=khoa)
        cboKhoa.grid(row = 8, column = 3, padx = 10, pady = 10,sticky=tk.W+tk.E)
        cboKhoa.set(dsKhoa[0])
        
        tk.Label(self, text = 'Địa chỉ liên hệ', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =9, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = DiaChi).grid(row =9, column=1,columnspan = 3, padx = 10, pady = 10,sticky = tk.W+tk.E)
  
        tk.Button(self,text="Clear",font= SMALLFONT,command=reset_item).grid(row = 10, column = 1, padx = 10, pady = 10)
        tk.Button(self,text="Lưu",font=SMALLFONT,command=add_item).grid(row = 10, column = 2, padx = 10, pady = 10)
        tk.Button(self,text="Xóa",font=SMALLFONT,command=remove_item).grid(row = 10, column = 3, padx = 10, pady = 10)
        tk.Button(self,text="Xuất file csv",font=SMALLFONT,command=ExportCSV).grid(row = 10, column = 4, padx = 10, pady = 10)

        
# trang thông tin môn học - MonHocPage
class MonHocPage(tk.Frame):
     
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        
        #khai báo biến
        MaMH = tk.StringVar()
        TenMH = tk.StringVar()
        TuKhoa = tk.StringVar()
        
        #Hàm hiển thị ds môn học lên lưới
        def show_item():  
            dsMH.delete(0,tk.END)
            dsMH.insert(tk.END, f'Id - Mã môn học - Tên môn học')
            dsMHFull = curѕor.execute("EXEC dbo.SP_LoadMonHoc ?, null", (TuKhoa.get()))
            for c in dsMHFull.fetchall():
                dsMH.insert(tk.END, f'{c[0]} - {c[1]} - {c[2]}')
        
        #hàm xóa dòng hiện tại trên lưới
        def remove_item():
            selection = dsMH.curselection()
            if len(selection) > 0:
                item = dsMH.get(selection[0])
                if item.split(' - ')[0] != 'Mã môn học':
                    res = messagebox.askquestion('Xác nhận', 
                                                 'Bạn chắc chắn muốn xóa môn học '+ item.split(' - ')[1] + ' - ' + item.split(' - ')[2] + '?')
                    if res == 'yes':
                        id = int(item.split(' - ')[0])
                        curѕor.execute("EXEC dbo.SP_XoaMonHoc ?", (id))
                        curѕor.commit()
                        show_item()
        
        #hàm thêm/cập nhật thông tin  môn học
        def add_item():
            if MaMH.get() == '' or TenMH.get() == '':
                messagebox.showinfo("Warning", "Vui lòng nhập đầy đủ mã/ tên môn học")
            else:
                curѕor.execute("EXEC dbo.SP_CapNhatMonHoc ?,?", (MaMH.get(),TenMH.get()))
                curѕor.commit()
                show_item()

        def reset_item():
            MaMH.set('')
            TenMH.set('')
         
        #hàm load thông tin dòng hiện tại lên các control
        def loadCurrent(event):
            if dsMH.curselection() != ():
                curMH = dsMH.get(dsMH.curselection()).split(' - ')
                if curMH[0].strip() !="Id":
                    mh = curѕor.execute("EXEC dbo.SP_LoadMonHoc null, ?", (curMH[0].strip())).fetchall()[0]
                    MaMH.set(mh[1])
                    TenMH.set(mh[2])
        
        #hiển thị 2 button chuyển đổi qua lại giữa 3 frame
        btnGV = tk.Button(self, text ="Quản lý Giảng viên",
                            command = lambda : controller.show_frame(GiangVienPage))
     
        btnGV.grid(row = 0, column = 0, padx = 10, pady = 10)
  
        btnGVMH = tk.Button(self, text ="Quản lý Giảng viên - môn học",
                            command = lambda : controller.show_frame(GiangVienMonHocPage))
     
        btnGVMH.grid(row = 0, column = 1, padx = 10, pady = 10)
        
        #tk.Button(self,text="Thêm mới",font= SMALLFONT,command=reset_item).grid(row = 0, column = 2, padx = 10, pady = 10,sticky = tk.W)
        
        #khai báo các control 
        tk.Label(self, text = 'Từ khóa (mã/tên MH)', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =2, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = TuKhoa).grid(row =2, column=1, columnspan=3,padx = 10, pady = 10,sticky = tk.W+tk.E)
        tk.Button(self,text="Tìm kiếm",font=SMALLFONT,command=show_item, justify=tk.LEFT, anchor="w").grid(row = 2, column = 4, padx = 10, pady = 10,sticky = tk.W)
       
        scroll_barMH = tk.Scrollbar(self, orient=tk.VERTICAL)
        dsMH = tk.Listbox(self, yscrollcommand=scroll_barMH.set, height=20)
        dsMH.grid(row = 3, column = 0, columnspan = 4, padx = 10, pady = 10,sticky = tk.W+tk.E)
        dsMH.bind("<<ListboxSelect>>", loadCurrent)       
        scroll_barMH.config(command=dsMH.yview)
        scroll_barMH.grid(row = 3, column = 4, padx = 10, pady = 10,sticky = tk.W+tk.E)
        
        tk.Label(self, text = 'Chi tiết thêm/ cập nhật', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =4, column = 0, padx = 10, pady = 10)
        ttk.Separator(self, orient=tk.HORIZONTAL).grid(row=4,column=1,columnspan=3, padx=10, pady=2, sticky=tk.EW)
        
        tk.Label(self, text = 'Mã môn học', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 5, column = 0, padx = 10, pady = 10)
        tk.Entry(self, textvariable = MaMH,width=70).grid(row = 5, column = 1, padx = 10, pady = 10,sticky = tk.E+tk.W)
        tk.Label(self, text = 'Tên môn học', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 5, column = 2, padx = 10, pady = 10)
        tk.Entry(self, textvariable = TenMH,width=70).grid(row = 5, column = 3, padx = 10, pady = 10,sticky = tk.W+tk.E)
        tk.Button(self,text="Lưu",font=SMALLFONT,command=add_item).grid(row = 6, column = 2, padx = 10, pady = 10,sticky = tk.W)
        tk.Button(self,text="Xóa",font=SMALLFONT,command=remove_item).grid(row = 6, column = 3, padx = 10, pady = 10, sticky = tk.W)


# trang thông tin giảng viên - môn học - GiangVienMonHocPage        
class GiangVienMonHocPage(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        #khai báo biến
        gv =tk.StringVar() 
        mh =tk.StringVar() 
        
        #Hàm hiển thị danh sách GV- MH
        def show_item():  
            dsGVMH.delete(0,tk.END)
            dsGVMH.insert(tk.END, f'Id - Mã GV - Họ tên GV - Mã môn học - Tên môn học')
            
            curGV = cboGV_Tim.get()
            curMH = cboMH_Tim.get()
            if curGV != '':
                idGV = ({i for i in lstDSGV if lstDSGV[i]==curGV}).pop()
            else:
                idGV = -1
            if curMH != '':
                idMH = ({i for i in lstDSMH if lstDSMH[i]==curMH}).pop()
            else:
                idMH = -1
            dsGVMHFull = curѕor.execute("EXEC dbo.SP_LoadGVMH ?,?,null", (idGV, idMH))
            for c in dsGVMHFull.fetchall():
                dsGVMH.insert(tk.END, f'{c[0]} - {c[1]} - {c[3]} {c[2]}: {c[5]}')
        
        #hàm xóa dòng hiện tại trên lưới
        def remove_item():
            selection = dsGVMH.curselection()
            if len(selection) > 0:
                item = dsGVMH.get(selection[0])
                if item.split(' - ')[0] != 'Id':
                    res = messagebox.askquestion('Xác nhận', 
                                                 'Bạn chắc chắn muốn xóa '+ item.split(' - ')[1] + ' - ' + item.split(' - ')[2] + '?')
                    if res == 'yes':
                        id = int(item.split(' - ')[0])
                        curѕor.execute("EXEC dbo.SP_XoaGVMH ?", (id))
                        curѕor.commit()
                        show_item()
        
        #Hàm thêm/cập nhật giảng viên - môn học
        def add_item():
            if cboGV.get() == '':
                messagebox.showinfo("Warning", "Vui lòng chọn giảng viên")
            elif cboMH.get() == '':
                messagebox.showinfo("Warning", "Vui lòng chọn môn học")
            else:
                IDGV= ({i for i in lstDSGV if lstDSGV[i]==cboGV.get()}).pop()
                IDMH = ({i for i in lstDSMH if lstDSMH[i]==cboMH.get()}).pop()
                curѕor.execute("EXEC dbo.SP_CapNhatGVMH ?,?", (IDGV,IDMH))
                curѕor.commit()
                show_item()
        
        def reset_item():
            gv.set('')
            mh.set('')
         
        #hàm load thông tin dòng hiện tại lên control
        def loadCurrent(event):
            if dsGVMH.curselection() != ():
                curGVMH = dsGVMH.get(dsGVMH.curselection()).split(' - ')
                if curGVMH[0].strip() !="Id":
                    gvmh = curѕor.execute("EXEC dbo.SP_LoadGVMH -1,-1,?", (curGVMH[0].strip())).fetchall()[0]
                    gv.set(f'{gvmh[1]} - {gvmh[3]} {gvmh[2]}')
                    mh.set(f'{gvmh[4]} - {gvmh[5]}')
        
        #hiển thị 2 button chuyển đổi qua lại giữa 3 frame
        btnMH = tk.Button(self, text ="Quản lý môn học",
                            command = lambda : controller.show_frame(MonHocPage))
     
        btnMH.grid(row = 0, column = 0, padx = 10, pady = 10)
  
        btnGV = tk.Button(self, text ="Quản lý Giảng viên",
                            command = lambda : controller.show_frame(GiangVienPage))
     
        btnGV.grid(row = 0, column = 1, padx = 10, pady = 10)
        
        #khai báo các control 
        tk.Label(self, text = 'Giảng viên', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 1, column = 0, padx = 10, pady = 10)
        
        #Load GV
        query="SELECT DISTINCT Id, GV = MaGiangVien + ' - ' + HoDem + ' ' + Ten FROM dbo.GiangVien"     
        lstGV =conn.execute(query)
        lstDSGV = {}
        for columnrow in lstGV.fetchall():
            lstDSGV[columnrow[0]] = columnrow[1]

        dsGV = list(lstDSGV.values())
        cboGV_Tim = ttk.Combobox(self, values=dsGV,width=70)
        cboGV_Tim.grid(row = 1, column = 1, padx = 10, pady = 10,sticky=tk.W+tk.E)

        #Load môn học
        tk.Label(self, text = 'Môn học', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 1, column = 2, padx = 10, pady = 10)
        queryMH="SELECT DISTINCT Id, MH = MaMonHoc + ' - ' + TenMonHoc FROM dbo.MonHoc"
        lstMH =conn.execute(queryMH)
        lstDSMH = {}
        for columnrow in lstMH.fetchall():
            lstDSMH[columnrow[0]] = columnrow[1]

        dsMH = list(lstDSMH.values())
        cboMH_Tim = ttk.Combobox(self, values=dsMH,width=70)
        cboMH_Tim.grid(row = 1, column = 3, padx = 10, pady = 10,sticky=tk.W+tk.E)
        
        tk.Button(self,text="Tìm kiếm",font=SMALLFONT,command=show_item, justify=tk.LEFT, anchor="w").grid(row = 1, column = 4, padx = 10, pady = 10,sticky = tk.W)
       
        scroll_barGVMH = tk.Scrollbar(self, orient=tk.VERTICAL,width=10)
        dsGVMH = tk.Listbox(self, yscrollcommand=scroll_barGVMH.set, height=20)
        scroll_barGVMH.config(command=dsGVMH.yview)
        scroll_barGVMH.grid(row = 2, column = 4, padx = 10, pady = 10,sticky = tk.W+tk.E)
        dsGVMH.grid(row = 2, column = 0, columnspan=4, padx = 10, pady = 10,sticky = tk.W+tk.E)
        dsGVMH.bind("<<ListboxSelect>>", loadCurrent)
        
        tk.Label(self, text = 'Chi tiết thêm/ cập nhật', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row =3, column = 0, padx = 10, pady = 10)
        ttk.Separator(self, orient=tk.HORIZONTAL).grid(row=3,column=1,columnspan=3, padx=10, pady=2, sticky=tk.EW)
        
        tk.Label(self, text = 'Giảng viên', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 4, column = 0, padx = 10, pady = 10)
        
        #query="SELECT DISTINCT Id, GV = MaGiangVien + ' - ' + HoDem + ' ' + Ten FROM dbo.GiangVien"
        #lstGV =conn.execute(query)
        #lstDSGV = {}
        #for columnrow in lstGV.fetchall():
            #lstDSGV[columnrow[0]] = columnrow[1]
        #dsGV = list(lstDSGV.values())
        cboGV = ttk.Combobox(self, values=dsGV, textvariable=gv,width=70)
        cboGV.grid(row = 4, column = 1, padx = 10, pady = 10,sticky=tk.W+tk.E)
        cboGV.set(dsGV[0])
        #Load môn học
        tk.Label(self, text = 'Môn học', font=SMALLFONT, justify=tk.LEFT, anchor="w").grid(sticky = tk.W,row = 4, column = 2, padx = 10, pady = 10)
        
        #queryMH="SELECT DISTINCT Id, MH = MaMonHoc + ' - ' + TenMonHoc FROM dbo.MonHoc"   
        #lstMH =conn.execute(queryMH)
        #lstDSMH = {}
        #for columnrow in lstMH.fetchall():
            #lstDSMH[columnrow[0]] = columnrow[1]
        #dsMH = list(lstDSMH.values())
        cboMH = ttk.Combobox(self, values=dsMH, textvariable=mh,width=70)
        cboMH.grid(row = 4, column = 3, padx = 10, pady = 10,sticky=tk.W+tk.E)        
        cboMH.set(dsMH[0])
        
        tk.Button(self,text="Lưu",font=SMALLFONT,command=add_item).grid(row = 5, column = 2, padx = 10, pady = 10)
        tk.Button(self,text="Xóa",font=SMALLFONT,command=remove_item).grid(row = 5, column = 3, padx = 10, pady = 10,sticky=tk.W)
# Execute Tkinter
root = baiCuoiKhoa()
root.mainloop()

In [15]:
#pip install unicodecsv